In [11]:
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import numpy as np

In [12]:
def scrape(*args):
    max_results_city = 120
    step = 10
    job_search = "data+analyst"
    if args:
        cities = set(args)
    else:
        cities = {"New York", "San Francisco"}

    city_search = [city.replace(" ", "+") for city in cities]
    city_search.sort()
    
    city_list = [city for city in cities] * max_results_city
    city_list.sort()
   
    url_list = get_url_list(city_search, job_search, max_results_city, step)

    results = scrape_links(url_list)
    results["search_city"] = city_list
    
    jobs_df = get_dataframe(results)
    # Save to new csv file
    output = os.path.join('..', '03_Data', 'Job_Data.csv')
    jobs_df.to_csv(output, header=True, index=True, index_label="Id", encoding="utf-8-sig")
    print("Done")
    data = jobs_df.to_dict(orient="records")
    insert_to_tables(data)
    return data

In [13]:
def get_url_list(city_search, job_search, max_results_city, step):
    url_list = []
    for city in city_search:
        for start in range(0, max_results_city, step):
            search_url = "http://www.indeed.com/jobs?q="+str(job_search)+"&l="+str(city)+"&start="+str(start)
            results_page = requests.get(search_url)
            #browser.visit(search_url)
            time.sleep(1)  #ensuring at least 1 second between page grabs
            #html = browser.html
            soup = BeautifulSoup(results_page.text, "lxml",)

            h2 = soup.find_all('h2', class_="jobtitle")
            for stuff in h2:
                href = stuff.find('a')['href']
                url = "http://www.indeed.com/"+str(href)
                url_list.append(url)
    return url_list

In [14]:
def scrape_links(url_list):
    job_company = []
    job_desc = []
    job_location = []
    job_title = []

    for url in url_list:
        job_page = requests.get(url,)
        time.sleep(1) 
        
        job_soup = BeautifulSoup(job_page.text, "lxml")
        
        title=company=location=job_summary="No information"
            
        title_object = job_soup.find("b", class_="jobtitle")
        if title_object is not None:
            title = title_object.text
        job_title.append(title)

        company_object = job_soup.find("span", class_="company")
        if company_object is not None:
            company = company_object.text
        job_company.append(company)

        location_object = job_soup.find("span", class_="location")
        if location_object is not None:
            location = location_object.text
        job_location.append(location)

        job_summary_object = job_soup.find("span", id="job_summary")
        if job_summary_object is not None:
            job_summary = job_summary_object.get_text()
            
        job_desc.append(job_summary)
        
    
    result = { "title":job_title,
               "description": job_desc, 
               "location": job_location,
               "company": job_company,
               "link" : url_list
             }

    return result

In [15]:
def get_dataframe(results):
    jobs_df = pd.DataFrame.from_dict(results)
    jobs_df = jobs_df.dropna(how='any')
    jobs_df = jobs_df[["title","company", "location","description", "search_city", "link"]]
    jobs_df.index = np.arange(1, len(jobs_df) + 1)
    return jobs_df

In [16]:
# Dependencies and boilerplate
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [17]:
Base = declarative_base()

In [18]:
# Define a JobPosition class
### BEGIN SOLUTION
class DataAnalyticsJob(Base):
    __tablename__ = "job_position"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    company = Column(String) 
    location = Column(String) 
    description = Column(Text) 
    search_city = Column(String) 
    link = Column(String) 
    
    def __repr__(self):
        return f"id={self.id}, title={self.title}"
### END SOLUTION

In [19]:
def insert_to_tables(data_result):
    # Create an engine to a SQLite database file
    #Unix/Mac - 4 initial slashes in total
    #engine = create_engine('sqlite:////absolute/path/to/foo.db')
    engine = create_engine("sqlite:///db/datanalyticsjobs.sqlite")
    # Create a connection to the engine called `conn`
    conn = engine.connect()
    # Use `create_all` to create the customers table in the database
    Base.metadata.create_all(engine)
    # Use MetaData from SQLAlchemy to reflect the tables
    ### BEGIN SOLUTION
    metadata = MetaData(bind=engine)
    metadata.reflect()
    ### END SOLUTION
    # Save the reference to the `customers` table as a variable called `table`
    table = sqlalchemy.Table('job_position', metadata, autoload=True)
    # Use `table.delete()` to remove any pre-existing data.
    # Note that this is a convenience function so that you can re-run the example code multiple times.
    # You would not likely do this step in production.
    conn.execute(table.delete())
    # Use `table.insert()` to insert the data into the table
    # The SQL table is populated during this step
    conn.execute(table.insert(), data_result)

In [20]:
data_result=scrape()

Done
